# KNN Multi-class Classification

## 1. Overview of KNN
K-Nearest Neighbors (KNN) is a **non-parametric supervised learning algorithm** used for classification and regression tasks.  
In **multi-class classification**, it predicts the class of a sample by majority voting among its $k$ nearest neighbors based on a chosen distance metric.

---

## 2. Workflow for KNN Multi-class Classification
1. **Data Preparation**:
   - Split the dataset into training and test sets.
   - Normalize or standardize features to ensure meaningful distance calculations.

2. **Choosing the Value of $k$**:
   - Select an appropriate $k$ (use cross-validation if needed).
   - Use an odd $k$ value to avoid ties.

3. **Distance Metric**:
   - Choose a distance metric (e.g., Euclidean, Manhattan).

4. **Prediction**:
   - Calculate distances between the test sample and all training samples.
   - Identify the $k$ nearest neighbors.
   - Assign the most common class among the neighbors.

5. **Evaluation**:
   - Use metrics such as accuracy, precision, recall, and F1-score to evaluate performance.

---

## 3. Evaluation Metrics
1. **Accuracy**:
   $$
   Accuracy = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}}
   $$

2. **Precision (Macro-Averaged)**:
   $$
   Precision_{\text{macro}} = \frac{1}{C} \sum_{c=1}^{C} Precision_c
   $$

3. **Recall (Macro-Averaged)**:
   $$
   Recall_{\text{macro}} = \frac{1}{C} \sum_{c=1}^{C} Recall_c
   $$

4. **F1 Score (Macro-Averaged)**:
   $$
   F1_{\text{macro}} = \frac{1}{C} \sum_{c=1}^{C} \frac{2 \cdot Precision_c \cdot Recall_c}{Precision_c + Recall_c}
   $$

---

## 4. Practical Tips
- **Feature Scaling**: Essential for meaningful distance calculations.
- **Optimal $k$**: A smaller $k$ is sensitive to noise, while a larger $k$ may smooth boundaries.
- **Class Imbalance**: Address imbalances by assigning weights or adjusting class distributions.
- **High-dimensional Data**: Performance can degrade in high dimensions (the curse of dimensionality); consider dimensionality reduction techniques like PCA.

---

## 5. Advantages and Disadvantages

### Advantages
- Simple and easy to implement.
- No assumptions about data distribution.
- Effective for small datasets.

### Disadvantages
- Computationally expensive, especially for large datasets.
- Sensitive to the choice of $k$ and distance metric.
- Struggles with high-dimensional data.

---

## 6. Conclusion
KNN is a straightforward and effective algorithm for **multi-class classification**. Its performance heavily depends on data preprocessing, distance metric selection, and $k$-value optimization. While suitable for small datasets, its limitations in computational efficiency and scalability make it less ideal for larger or more complex datasets. For such cases, consider alternative algorithms like Random Forest or SVM.


## 1. Prepare Data

In [ ]:
!wget https://raw.githubusercontent.com/devdio/flyai_datasets/main/iris.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

SEED = 42

In [ ]:
# iris_link = 'https://raw.githubusercontent.com/devdio/flyai_datasets/main/iris.csv'
# iris = pd.read_csv(iris_link)
iris = pd.read_csv('iris.csv')
iris.shape

In [ ]:
# Create a dictionary
# where each column name in the iris DataFrame is converted
# to lowercase and spaces are replaced with underscores.

col_dict = {col: col.lower().replace(' ', '_' ) for col in iris.columns}
col_dict

In [ ]:
iris.rename(columns=col_dict, inplace=True)

In [ ]:
# Missing
iris.isna().sum(axis=0)

# Duplication
iris.duplicated().sum()

In [ ]:
# Split into training and testing datasets
iris = iris.sample(frac=1, random_state=SEED)
iris.head()

In [ ]:
idx_train = int(len(iris) * 0.6)
idx_val = int(len(iris)* 0.8)
idx_train, idx_val

In [ ]:
train = iris.iloc[:idx_train, :]
val = iris.iloc[idx_train:idx_val, :]
test = iris.iloc[idx_val:, :]

train.shape, val.shape, test.shape

In [ ]:
X_train = train.drop('species', axis=1)
y_train = train['species']

X_val = val.drop('species', axis=1)
y_val = val['species']

y_train.value_counts(), y_val.value_counts()

In [ ]:
u = X_train.mean()
std = X_train.std()

X_train_s = (X_train - u)/std
X_train_s.head()

In [ ]:
X_val_s = (X_val - u)/std
X_val_s.head()

In [ ]:
ss_dic = {'mean':u, 'std':std}
ss_dic

In [ ]:
label_dict = {specie:i  for i, specie in enumerate(y_val.unique())}
label_dict

In [ ]:
y_train_e = y_train.map(label_dict)
y_val_e = y_val.map(label_dict)

y_train_e, y_val_e

In [ ]:
X_train_s = X_train_s.to_numpy()
y_train_e = y_train_e.to_numpy()

X_val_s = X_val_s.to_numpy()
y_val_e = y_val_e.to_numpy()

print(X_train_s.shape, y_train_e.shape)
print(X_val_s.shape, y_val_e.shape)
print(type(X_train_s), type(y_train_e))
print(type(X_val_s), type(y_val_e))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train_s, y_train_e)

y_pred = clf.predict(X_val_s)
y_pred

In [ ]:
(y_pred == y_val_e).sum()/len(y_val_e)

In [ ]:
from sklearn.metrics import accuracy_score

scores = []
for k in range(3, 30):
    clf = KNeighborsClassifier(n_neighbors=k)
    clf.fit(X_train_s, y_train_e)
    y_pred = clf.predict(X_val_s)
    acc = accuracy_score(y_val_e, y_pred)
    scores.append(acc)

In [ ]:
plt.plot(scores)

In [ ]:
X_test = test.drop('species', axis=1)
y_test = test['species']

X_test_s = (X_test - ss_dic['mean'])/ss_dic['std']
y_test_e = y_test.map(label_dict)

X_test_s = X_test_s.to_numpy()
y_test_e = y_test_e.to_numpy()

y_pred = clf.predict(X_test_s)

In [ ]:
(y_test_e == y_pred).sum()/len(y_test_e)

In [ ]:
from sklearn.metrics import confusion_matrix

cfm = confusion_matrix(y_test_e, y_pred)
cfm

In [ ]:
s = sns.heatmap(cfm, annot=True, cmap='Blues', fmt='d', cbar=False)
s.set(xlabel='Prediction', ylabel='Actual')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score

print('accuracy:', accuracy_score(y_test_e, y_pred))
print('recall:', recall_score(y_test_e, y_pred, average='macro'))
print('precision:', precision_score(y_test_e, y_pred, average='macro'))
print('f1 :', f1_score(y_test_e, y_pred, average='macro'))